In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
df = pd.read_csv("/content/spotify_millsongdata.csv")
df.head()

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [3]:
df = df[0:50000]

In [4]:
df.drop(["link"], axis = 1, inplace = True)
df.rename(columns = {"text":"lyrics"}, inplace = True)
df.drop_duplicates(subset="song", inplace=True)
df.reset_index(drop = True, inplace = True)
df

,artist,song,lyrics
0,ABBA,Ahe's My Kind Of Girl,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante","Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,Making somebody happy is a question of give an...
...,...,...,...
39461,Ramones,Don't Bust My Chops,I'm sick and tired of you calling me names \r...
39462,Ramones,Don't Come Close,You know it's generally known \r\nYou got eve...
39463,Ramones,Don't Pour Out All The Oil,Verse: \r\nDon't pour out all the oil \r\nIt...
39464,Ramones,Eat That Rat,One-two-three-four \r\n \r\nYou don't want t...


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39466 entries, 0 to 39465
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   artist  39466 non-null  object
 1   song    39466 non-null  object
 2   lyrics  39466 non-null  object
dtypes: object(3)
memory usage: 925.1+ KB


In [6]:
df["combined_features"] = df["artist"]+ " "+df["song"]+" "+df["lyrics"]
df

,artist,song,lyrics,combined_features
0,ABBA,Ahe's My Kind Of Girl,"Look at her face, it's a wonderful face \r\nA...","ABBA Ahe's My Kind Of Girl Look at her face, i..."
1,ABBA,"Andante, Andante","Take it easy with me, please \r\nTouch me gen...","ABBA Andante, Andante Take it easy with me, pl..."
2,ABBA,As Good As New,I'll never know why I had to go \r\nWhy I had...,ABBA As Good As New I'll never know why I had ...
3,ABBA,Bang,Making somebody happy is a question of give an...,ABBA Bang Making somebody happy is a question ...
4,ABBA,Bang-A-Boomerang,Making somebody happy is a question of give an...,ABBA Bang-A-Boomerang Making somebody happy is...
...,...,...,...,...
39461,Ramones,Don't Bust My Chops,I'm sick and tired of you calling me names \r...,Ramones Don't Bust My Chops I'm sick and tired...
39462,Ramones,Don't Come Close,You know it's generally known \r\nYou got eve...,Ramones Don't Come Close You know it's general...
39463,Ramones,Don't Pour Out All The Oil,Verse: \r\nDon't pour out all the oil \r\nIt...,Ramones Don't Pour Out All The Oil Verse: \r\...
39464,Ramones,Eat That Rat,One-two-three-four \r\n \r\nYou don't want t...,Ramones Eat That Rat One-two-three-four \r\n ...


In [7]:
df.combined_features[0]

"ABBA Ahe's My Kind Of Girl Look at her face, it's a wonderful face  \r\nAnd it means something special to me  \r\nLook at the way that she smiles when she sees me  \r\nHow lucky can one fellow be?  \r\n  \r\nShe's just my kind of girl, she makes me feel fine  \r\nWho could ever believe that she could be mine?  \r\nShe's just my kind of girl, without her I'm blue  \r\nAnd if she ever leaves me what could I do, what could I do?  \r\n  \r\nAnd when we go for a walk in the park  \r\nAnd she holds me and squeezes my hand  \r\nWe'll go on walking for hours and talking  \r\nAbout all the things that we plan  \r\n  \r\nShe's just my kind of girl, she makes me feel fine  \r\nWho could ever believe that she could be mine?  \r\nShe's just my kind of girl, without her I'm blue  \r\nAnd if she ever leaves me what could I do, what could I do?\r\n\r\n"

In [8]:
for i in range(0,5):
    print("*********************************************\n")
    print(df.combined_features[i])

*********************************************

ABBA Ahe's My Kind Of Girl Look at her face, it's a wonderful face  
And it means something special to me  
Look at the way that she smiles when she sees me  
How lucky can one fellow be?  
  
She's just my kind of girl, she makes me feel fine  
Who could ever believe that she could be mine?  
She's just my kind of girl, without her I'm blue  
And if she ever leaves me what could I do, what could I do?  
  
And when we go for a walk in the park  
And she holds me and squeezes my hand  
We'll go on walking for hours and talking  
About all the things that we plan  
  
She's just my kind of girl, she makes me feel fine  
Who could ever believe that she could be mine?  
She's just my kind of girl, without her I'm blue  
And if she ever leaves me what could I do, what could I do?


*********************************************

ABBA Andante, Andante Take it easy with me, please  
Touch me gently like a summer evening breeze  
Take your time, m

In [9]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
def cleaning(text):
    text = re.sub("[^a-zA-Z]", " ", text)
    text = text.lower()
    text = text.split()
    stops = set(stopwords.words("english"))
    text = [w for w in text if not w in stops]
    text = " ".join(text)
    return text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
df["combined_features"] = df["combined_features"].apply(func = cleaning)

In [11]:
for i in range(0,5):
    print("\n*********************************************\n")
    print(df.combined_features[i])


*********************************************

abba ahe kind girl look face wonderful face means something special look way smiles sees lucky one fellow kind girl makes feel fine could ever believe could mine kind girl without blue ever leaves could could go walk park holds squeezes hand go walking hours talking things plan kind girl makes feel fine could ever believe could mine kind girl without blue ever leaves could could

*********************************************

abba andante andante take easy please touch gently like summer evening breeze take time make slow andante andante let feeling grow make fingers soft light let body velvet night touch soul know andante andante go slowly music music song song music song play time time make strong play cause making strong make sing make sound make sing make andante andante tread lightly ground andante andante oh please let shimmer eyes like feeling thousand butterflies please talk go play andante andante let float away music music song 

In [12]:
df.combined_features[0]

'abba ahe kind girl look face wonderful face means something special look way smiles sees lucky one fellow kind girl makes feel fine could ever believe could mine kind girl without blue ever leaves could could go walk park holds squeezes hand go walking hours talking things plan kind girl makes feel fine could ever believe could mine kind girl without blue ever leaves could could'

In [13]:
tfv = TfidfVectorizer(max_features = 10000)
tfv_matrix = tfv.fit_transform(df["combined_features"])
tfv_matrix.shape

(39466, 10000)

In [ ]:
cosine_sim = cosine_similarity(tfv_matrix)

In [14]:
song_user_likes = "Hope"
song_index = df[df.song == song_user_likes].index[0]

In [15]:
song_index

659

In [ ]:
similar_songs = list(enumerate(cosine_sim[song_index]))

In [ ]:
similar_songs[0:5]

In [ ]:
sorted_similar_songs = sorted(similar_songs, key=lambda x:x[1], reverse=True)

In [ ]:
sorted_similar_songs[0:5]

In [ ]:
for song in sorted_similar_songs[1:11]:
        similar_songs = df[df.index == song[0]]["song"].values[0]
        print(similar_songs)

In [ ]:
def recommended_songs(song_user_likes):
    song_index = df[df.song == song_user_likes].index[0]
    similar_songs = list(enumerate(cosine_sim[song_index]))
    sorted_similar_songs = sorted(similar_songs, key=lambda x:x[1], reverse=True)
    
    for song in sorted_similar_songs[1:11]:
        similar_songs = df[df.index == song[0]]["song"].values[0]
        print(similar_songs)

In [ ]:
recommended_songs("Hope")